# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [6]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

from IPython.display import clear_output
clear_output(wait=True)

# Import API key
from api_keys import geoapify_key


In [7]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tasiilaq,65.6145,-37.6368,20.82,91,65,4.61,GL,1681441030
1,1,panlaitan,12.1191,119.8486,82.47,74,100,4.56,PH,1681441089
2,2,tuktoyaktuk,69.4541,-133.0374,-2.20,71,20,5.75,CA,1681441089
3,3,mataura,-46.1927,168.8643,63.81,59,0,7.92,NZ,1681441089
4,4,manta,-0.9500,-80.7333,80.38,89,20,6.91,EC,1681440308


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng"
    ,"Lat"
    ,geo = True
    ,tiles = "OSM"
    ,frame_width = 800
    ,frame_height = 600
    ,size = "Humidity"
    ,color = "City"
)

# Display the map plot
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
sample_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 85) 
                                    & (city_data_df["Max Temp"] > 70) 
                                    & (city_data_df["Wind Speed"] < 6) 
                                    & (city_data_df["Cloudiness"] == 0)]


# Drop any rows with null values
sample_weather_df = sample_weather_df.dropna()

# Display sample data
sample_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,bethel,41.3712,-73.4140,73.29,41,0,1.99,US,1681441090
70,70,sterling,39.0062,-77.4286,72.12,47,0,3.44,US,1681440887
111,111,atkinson,42.8384,-71.1470,75.38,52,0,5.75,US,1681440899
118,118,quatre cocos,-20.2078,57.7625,76.08,84,0,1.12,MU,1681441116
210,210,ginda,30.6945,78.4932,72.72,20,0,0.76,IN,1681441132
308,308,palmer,42.1584,-72.3287,71.92,59,0,3.44,US,1681441148
311,311,saint-pierre,-21.3393,55.4781,76.68,73,0,5.75,RE,1681441148
319,319,tottori,35.5000,134.2333,75.27,29,0,5.75,JP,1681441149
320,320,iranshahr,27.2025,60.6848,77.34,47,0,4.61,IR,1681441150
348,348,panzhihua,26.5851,101.7128,82.20,21,0,1.14,CN,1681441156


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = sample_weather_df[["City", "Country", "Lat","Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
6,bethel,US,41.3712,-73.4140,41,
70,sterling,US,39.0062,-77.4286,47,
111,atkinson,US,42.8384,-71.1470,52,
118,quatre cocos,MU,-20.2078,57.7625,84,
210,ginda,IN,30.6945,78.4932,20,
308,palmer,US,42.1584,-72.3287,59,
311,saint-pierre,RE,-21.3393,55.4781,73,
319,tottori,JP,35.5000,134.2333,29,
320,iranshahr,IR,27.2025,60.6848,47,
348,panzhihua,CN,26.5851,101.7128,21,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
from urllib.parse import urlencode
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    #conbine_url = base_url + "?" + urlencode(params)
    #print(conbine_url)

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
     # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df 

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
sterling - nearest hotel: Residence Inn by Marriott Dulles Airport at Dulles 28 Centre
atkinson - nearest hotel: Best Western Plus Merrimack Valley
quatre cocos - nearest hotel: Horisun
ginda - nearest hotel: Shivalinga
palmer - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
tottori - nearest hotel: とらや旅館
iranshahr - nearest hotel: هتل قصِر
panzhihua - nearest hotel: 木棉花酒店
lakeway - nearest hotel: Lakeway Resort and Spa
gunjur - nearest hotel: Footsteps Eco Lodge


,City,Country,Lat,Lng,Humidity,Hotel Name
6,bethel,US,41.3712,-73.4140,41,Hampton Inn Danbury
70,sterling,US,39.0062,-77.4286,47,Residence Inn by Marriott Dulles Airport at Du...
111,atkinson,US,42.8384,-71.1470,52,Best Western Plus Merrimack Valley
118,quatre cocos,MU,-20.2078,57.7625,84,Horisun
210,ginda,IN,30.6945,78.4932,20,Shivalinga
308,palmer,US,42.1584,-72.3287,59,No hotel found
311,saint-pierre,RE,-21.3393,55.4781,73,Tropic Hotel
319,tottori,JP,35.5000,134.2333,29,とらや旅館
320,iranshahr,IR,27.2025,60.6848,47,هتل قصِر
348,panzhihua,CN,26.5851,101.7128,21,木棉花酒店


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
#Clear Cashe memory
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng"
    ,"Lat"
    ,geo=True
    ,tiles="OSM"
    ,frame_width=800
    ,frame_height=600
    ,size=["Humidity", "Hotel Name", "Country"]
    ,color="City"
)



# Display the map
map_plot_2